# Run carpel detection on multiple x-ray images

## Set directories

In [ ]:
# make empty dataframe to store carpel area info
all_dfs = pd.DataFrame()

# set directory to folder with x-ray images. Remember to convert from tif to 8-bit RGB jpeg
# to-do: add batch tif to jpeg conversion
dataset_dir = (ROOT_DIR + "/datasets/21UM/gt1P")

# set save directory for mask arrays (if desired) and final csv
save_dir = (ROOT_DIR + "/results/21UM/gt1P")
os.mkdir(save_dir + "/mask_arrays")

dataset_all = os.listdir(dataset_dir)
# dataset = fnmatch.filter(dataset_all, '*08.jpg') + fnmatch.filter(dataset_all, '*10.jpg') # if desired, modify which x-ray image angles are included in the analysis
dataset = dataset_all
print("Measuring " + str(len(dataset)) + " images from " + dataset_dir)
print("Saving output in " + save_dir)

## Calculate carpel area  

In [ ]:
# detect carpels in each x-ray
# inputs: x-ray jpegs from dataset_dir
# outputs: dataframe all_dfs with all carpel area measurements. Written to carpel_area_results.csv in save_dir
for image_id in dataset:

    #generate temp dataframe for image_id
    df = pd.DataFrame(area_data, columns=['image_id', 'carpel_number', 'carpel_area'])
    df["image_id"] = str(image_id)
    
    #run detection
    print("Running detection on image " + str(dataset.index(image_id) + 1) + "/" + str(len(dataset)))
    image = skimage.io.imread(dataset_dir + '/' + image_id)
    r = model.detect([image], verbose=0)
    # np.save(save_dir + "/mask_arrays/" + str(image_id) + ".npy", r) # if desired, save numpy array
    r = r[0]
    
    # skip image if no carpels detected
    if np.size(r['masks']) == 0:
        print("No carpels detected.")
        continue
        
    #plot the detections
    ax = get_ax(1)
    display_carpel_instances(image, r['rois'], r['masks'], r['class_ids'], r['scores'], ax=ax, title= image_id + " Predictions")
    
    #if desired, save mask overlay plot of detected carpels
    #plt.savefig(save_dir + "/" + str(image_id) + '.masks.jpg')
    #plt.close()
    
    #measure carpel pixel area
    carpel_count = np.reshape(r['masks'], (-1, r['masks'].shape[-1])).astype(np.float32).shape
    print(str(carpel_count[1]) + " carpels detected.")
    area_data = np.ndarray(shape=(carpel_count[1],3), dtype=float, order='F')
    for x in range(0,(carpel_count[1])):
        area_sum = np.reshape(r['masks'], (-1, r['masks'].shape[-1])).astype(np.float32)[:,x].sum()
        print("Pixel area of carpel " + str(x) + ": " + str(area_sum))
        df["carpel_number"][x] = x
        df["carpel_area"][x] = area_sum
    
    #merge new dataframe with previous dataframe
    all_dfs = pd.concat([all_dfs, df])
    #write dataframe to csv at each step
    all_dfs.to_csv(save_dir + "/carpel_area_results.csv", header= True)